In [1]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 모듈 모음
import modules as mds

In [99]:
# 누적 설비용량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total = pd.read_excel("dataset/지역별_누적_설비용량.xlsx")
data_2024 = pd.read_excel("dataset/2024년도_신규_설비용량.xlsx")
data_2023 = pd.read_excel("dataset/2023년도_신규_설비용량.xlsx")
data_2022 = pd.read_excel("dataset/2022년도_신규_설비용량.xlsx")
data_2021 = pd.read_excel("dataset/2021년도_신규_설비용량.xlsx")

# 데이터프레임으로 변환
# ic = Installed Capacity = 설비용량
ic_total = pd.DataFrame(data_total)
ic_2024 = pd.DataFrame(data_2024)
ic_2023 = pd.DataFrame(data_2023)
ic_2022 = pd.DataFrame(data_2022)
ic_2021 = pd.DataFrame(data_2021)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(ic_total)
mds.delete_comma(ic_2024)
mds.delete_comma(ic_2023)
mds.delete_comma(ic_2022)
mds.delete_comma(ic_2021)

# 컴럼 통일화를 위해 capacity_total에 "합계" 컬럼 추가
ic_total["합계"] = ic_total.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
ic_total = pd.concat([ic_total, ic_2024, ic_2023, ic_2022, ic_2021], ignore_index=True)

# 각 연도별 누적값 계산
for i in range(1, len(ic_total)):
    ic_total.iloc[i, 1:] = ic_total.iloc[i-1, 1:] - ic_total.iloc[i, 1:]

# 연도 정리
ic_total.rename(columns={"Unnamed: 0": "연도"}, inplace=True)
ic_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 오름차순 정렬 (주석 처리 시 내림차순)
# ic_total.sort_values(by="연도", inplace=True)
# ic_total.reset_index(drop=True, inplace=True)

# 2020~2024 각 연도의 지역별 누적 설비용량
ic_total.head()

ic_total.to_excel("dataset/지역별_연도별_설비용량.xlsx", index=False, engine="openpyxl")


In [100]:
# 누적 발전량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total_2022 = pd.read_excel("dataset/지역별_누적_발전량.xlsx")
data_2024 = pd.read_excel("dataset/2024년_신규_발전량.xlsx")
data_2023 = pd.read_excel("dataset/2023년_신규_발전량.xlsx")
data_2022 = pd.read_excel("dataset/2022년_신규_발전량.xlsx")
data_2021 = pd.read_excel("dataset/2021년_신규_발전량.xlsx")

# 데이터프레임으로 변환
# gen = Power Generation = 발전량
gen_total_2022 = pd.DataFrame(data_total_2022)
gen_2024 = pd.DataFrame(data_2024)
gen_2023 = pd.DataFrame(data_2023)
gen_2022 = pd.DataFrame(data_2022)
gen_2021 = pd.DataFrame(data_2021)

# 열과 행 전치
gen_2024 = gen_2024.T.reset_index(drop=True)
gen_2023 = gen_2023.T.reset_index(drop=True)
gen_2022 = gen_2022.T.reset_index(drop=True)
gen_2021 = gen_2021.T.reset_index(drop=True)

# 컴럼 정리
gen_2024.columns = gen_2024.iloc[0]
gen_2024.drop(labels=gen_2024.index[0], axis=0, inplace=True)
gen_2023.columns = gen_2023.iloc[0]
gen_2023.drop(labels=gen_2023.index[0], axis=0, inplace=True)
gen_2022.columns = gen_2022.iloc[0]
gen_2022.drop(labels=gen_2022.index[0], axis=0, inplace=True)
gen_2021.columns = gen_2021.iloc[0]
gen_2021.drop(labels=gen_2021.index[0], axis=0, inplace=True)


# 연도 컬럼 추가
gen_2024.insert(0, "연도", np.nan)
gen_2023.insert(0, "연도", np.nan)
gen_2022.insert(0, "연도", np.nan)
gen_2021.insert(0, "연도", np.nan)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(gen_total_2022)
mds.delete_comma(gen_2024)
mds.delete_comma(gen_2023)
mds.delete_comma(gen_2022)
mds.delete_comma(gen_2021)

# 월별 데이터 연도별 데이터로 변환
gen_2024 = pd.DataFrame(gen_2024.sum()).T
gen_2023 = pd.DataFrame(gen_2023.sum()).T
gen_2022 = pd.DataFrame(gen_2022.sum()).T
gen_2021 = pd.DataFrame(gen_2021.sum()).T

# "합계" 컬럼 추가
gen_2024["합계"] = gen_2024.sum(axis=1, numeric_only=True)
gen_2023["합계"] = gen_2023.sum(axis=1, numeric_only=True)
gen_2022["합계"] = gen_2022.sum(axis=1, numeric_only=True)
gen_2021["합계"] = gen_2021.sum(axis=1, numeric_only=True)

# # 하나의 DataFrame으로 병합
gen_total = pd.concat([gen_2024, gen_2023, gen_total_2022, gen_2022, gen_2021], ignore_index=True)

# 연도 정리
gen_total = gen_total.drop(gen_total.columns[-1], axis=1)
gen_total.rename(columns={"지역": "연도"}, inplace=True)
gen_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 소수점 제거
gen_total = gen_total.astype(int)

# # 각 연도별 누적값 계산
gen_total.iloc[1, 1:] = gen_total.iloc[2, 1:] + gen_total.iloc[1, 1:] # 2023 계산
gen_total.iloc[0, 1:] = gen_total.iloc[1, 1:] + gen_total.iloc[0, 1:] # 2024 계산
gen_total.iloc[3, 1:] = gen_total.iloc[2, 1:] - gen_total.iloc[3, 1:] # 2021 계산
gen_total.iloc[4, 1:] = gen_total.iloc[3, 1:] - gen_total.iloc[4, 1:] # 2020 계산


# # 오름차순 정렬 (주석 처리 시 내림차순)
# # gen_total.sort_values(by="연도", inplace=True)
# # gen_total.reset_index(drop=True, inplace=True)

# # 2020~2024 각 연도의 지역별 누적 발전량
gen_total.head()

gen_total.to_excel("dataset/지역별_연도별_발전량.xlsx", index=False, engine="openpyxl")

In [101]:
import pandas as pd

# 엑셀 불러오기
capacity = pd.read_excel("dataset/지역별_연도별_설비용량.xlsx")
generation = pd.read_excel("dataset/지역별_연도별_신규_발전량.xlsx")

# 설비용량 long 변환 (합계 제외)
cap_long = capacity.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="설비용량(MW)"
)

# 발전량 long 변환 (합계 제외)
gen_long = generation.drop(columns=["합계"]).melt(
    id_vars=["연도"], var_name="지역", value_name="발전량(MWh)"
)

# 데이터 합치기
merged = pd.merge(cap_long, gen_long, on=["연도", "지역"])

# 설비 이용률 계산
merged["설비이용률(%)"] = merged["발전량(MWh)"] / (merged["설비용량(MW)"] * 8760) * 100

# 연도 기준 내림차순 (2024 → 2020)
merged_sorted = merged.sort_values(by="연도", ascending=False)

# 인덱스 초기화
merged_sorted = merged_sorted.reset_index(drop=True)

print(merged_sorted.head(20))  # 상위 20개 확인

merged_sorted.to_excel("dataset/합친데이터.xlsx", index=False, engine="openpyxl")


      연도  지역  설비용량(MW)  발전량(MWh)   설비이용률(%)
0   2024  서울        50     57372  13.098630
1   2024  강원      1943   2183283  12.827237
2   2024  경남      2016   2203811  12.478998
3   2024  경북      3842   4340591  12.896961
4   2024  대전        65     66946  11.757288
5   2024  인천       163    163075  11.420778
6   2024  제주       621    634506  11.663799
7   2024  전남      6023   7119186  13.493151
8   2024  대구       420    443426  12.052240
9   2024  경기      2012   2065997  11.721889
10  2024  부산       275    287942  11.952760
11  2024  전북      4590   5579509  13.876476
12  2024  충남      3772   4299136  13.010842
13  2024  충북      1613   1747500  12.367409
14  2024  광주       353    360149  11.646714
15  2023  광주       303    307606  11.589056
16  2023  경기      1672   1665112  11.368498
17  2023  전남      5410   6382965  13.468558
18  2023  대전        55     55787  11.578871
19  2023  충남      3072   3485642  12.952615


In [102]:
import pandas as pd
import numpy as np
import plotly.express as px

# === 0) 데이터 정리: '연도' + 지역 컬럼들만 평균 ===
df = gen_total.copy()
if df.columns[0] != "연도":
    df = df.rename(columns={df.columns[0]: "연도"})

# '합계'가 있으면 제외
area_cols = [c for c in df.columns if c not in ["연도", "합계"]]

# 숫자형 보정
for c in area_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 연도별 지역 평균
year_mean_df = pd.DataFrame({
    "연도": df["연도"],
    "평균 발전량(MWh)": df[area_cols].mean(axis=1).round()
}).sort_values("연도")

# === 1) Plotly 막대 ===
fig = px.bar(
    year_mean_df, x="연도", y="평균 발전량(MWh)",
    title="연도별 평균 태양광 발전량(MWh)",
    text="평균 발전량(MWh)",
    template="plotly_white"
)

# 보기 좋게 라벨/축/크기
fig.update_traces(texttemplate="%{text:,.0f}", textposition="outside")
fig.update_layout(
    width=1200, height=800,
    title_x=0.5, title_y=0.96,
    title_font_size=25, title_font_color="black",
    margin=dict(l=40, r=40, t=80, b=60)
)
fig.update_yaxes(title_text="평균 발전량(MWh)", tickformat=",.0f")
fig.update_xaxes(title_text="연도")

fig.show()
# fig.write_html("year_mean_bar.html", include_plotlyjs="cdn")


In [173]:
import pandas as pd
import plotly.graph_objects as go

# 데이터 로드
df = pd.read_excel("dataset/일사량합친데이터.xlsx")

# 2024년만 선택
df_2024 = df[df["연도"] == 2024].copy()

# 발전량 그래프
fig_gen = go.Figure()
fig_gen.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["발전량(MWh)"],
    text=[f"{v:,.0f}" for v in df_2024["발전량(MWh)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["발전량(MWh)"],
        colorscale="Blues",
        showscale=True,
        colorbar=dict(title="MWh")
    ),
    hovertemplate="<b>%{x}</b><br>발전량: %{y:,.0f} MWh<extra></extra>"
))
fig_gen.update_layout(
    title="2024년 지역별 발전량",
    xaxis_title="지역",
    yaxis_title="발전량 (MWh)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_gen.update_xaxes(tickangle=-45)
fig_gen.update_yaxes(tickformat=",.0f")

# 설비용량 그래프
fig_cap = go.Figure()
fig_cap.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["설비용량(MW)"],
    text=[f"{v:,.0f}" for v in df_2024["설비용량(MW)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["설비용량(MW)"],
        colorscale="Greens",
        showscale=True,
        colorbar=dict(title="MW")
    ),
    hovertemplate="<b>%{x}</b><br>설비용량: %{y:,.0f} MW<extra></extra>"
))
fig_cap.update_layout(
    title="2024년 지역별 설비용량",
    xaxis_title="지역",
    yaxis_title="설비용량 (MW)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_cap.update_xaxes(tickangle=-45)
fig_cap.update_yaxes(tickformat=",.0f")

# 결과 출력
fig_gen.show()
fig_cap.show()

In [106]:
import pandas as pd

# === 1) 데이터 불러오기 ===
df_main = pd.read_excel("dataset/합친데이터.xlsx")
df_solar = pd.read_excel("dataset/연도별 일사량 데이터.xlsx")

# === 2) 가로형 → 세로형 변환 ===
df_solar_long = df_solar.melt(
    id_vars="지역", 
    var_name="연도", 
    value_name="일사량(MJ/m2)"
)

# 연도 숫자형으로 변환
df_solar_long["연도"] = df_solar_long["연도"].astype(int)

# === 3) 두 데이터 합치기 ===
merged = pd.merge(
    df_main,
    df_solar_long,
    on=["연도", "지역"],
    how="left"
)

merged["일사량(MJ/m2)"] = merged["일사량(MJ/m2)"] / 12

print(merged)

merged.to_excel("dataset/최종일사량합친데이터.xlsx", index=False, engine="openpyxl")

      연도  지역  설비용량(MW)  발전량(MWh)   설비이용률(%)  일사량(MJ/m2)
0   2024  서울        50     57372  13.098630  436.333333
1   2024  강원      1943   2183283  12.827237  408.083333
2   2024  경남      2016   2203811  12.478998  465.833333
3   2024  경북      3842   4340591  12.896961  462.583333
4   2024  대전        65     66946  11.757288  451.916667
..   ...  ..       ...       ...        ...         ...
70  2020  광주       215    225188  11.956462  459.500000
71  2020  대구       200    191816  10.948402  463.083333
72  2020  경남      1085   1085723  11.423132  462.250000
73  2020  충북       846    871117  11.754442  447.666667
74  2020  전북      2559   2881921  12.856054  432.750000

[75 rows x 6 columns]


In [170]:
import plotly.graph_objects as go

# === 지역 필터링 & 정리 ===
target_regions = ["경북", "경남", "충남"]
df_filtered = merged[merged["지역"].isin(target_regions)].copy()

df_filtered["연도"] = pd.to_numeric(df_filtered["연도"], errors="coerce")
df_grouped = (
    df_filtered.groupby(["연도", "지역"])[["발전량(MWh)", "설비용량(MW)"]]
    .sum()
    .reset_index()
)

# === 색상 고정 ===
color_map = {
    "경남": "#1f77b4",  # 파랑
    "경북": "#d62728",  # 빨강
    "충남": "#2ca02c"   # 초록
}

# ====================================================
# 1) 연도별 발전량 추이 (라인 차트)
# ====================================================
fig_gen = go.Figure()

for region in target_regions:
    df_region = df_grouped[df_grouped["지역"] == region]
    fig_gen.add_trace(
        go.Scatter(
            x=df_region["연도"],
            y=df_region["발전량(MWh)"],
            mode="lines+markers",
            name=region,
            line=dict(color=color_map[region], width=3),
            marker=dict(size=9, symbol="circle")
        )
    )

fig_gen.update_layout(
    title="경북·경남·충남 연도별 발전량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="발전량(MWh)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)",
                bordercolor="lightgray", borderwidth=1)
)

fig_gen.show()

# ====================================================
# 2) 연도별 설비용량 추이 (라인 차트)
# ====================================================
fig_cap = go.Figure()

for region in target_regions:
    df_region = df_grouped[df_grouped["지역"] == region]
    fig_cap.add_trace(
        go.Scatter(
            x=df_region["연도"],
            y=df_region["설비용량(MW)"],
            mode="lines+markers",
            name=region,
            line=dict(color=color_map[region], width=3, dash="dot"),  # 점선
            marker=dict(size=8, symbol="square")
        )
    )

fig_cap.update_layout(
    title="경북·경남·충남 연도별 설비용량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="설비용량(MW)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)",
                bordercolor="lightgray", borderwidth=1)
)

fig_cap.show()


In [134]:
import plotly.express as px

# === 1) 지역 필터링 ===
target_regions = ["경북", "경남", "충남"]
df_filtered = merged[merged["지역"].isin(target_regions)].copy()

# === 2) 연도 숫자형 변환 + 정렬 ===
df_filtered["연도"] = pd.to_numeric(df_filtered["연도"], errors="coerce")
df_filtered = df_filtered.sort_values("연도")

# === 3) 연도별 일사량 평균 계산 (지역별) ===
df_grouped = (
    df_filtered.groupby(["연도", "지역"])["일사량(MJ/m2)"]
    .mean()
    .reset_index()
)

# === 4) Plotly 선그래프 ===
fig = px.line(
    df_grouped,
    x="연도",
    y="일사량(MJ/m2)",
    color="지역",
    markers=True,
    title="경북·경남·충남 연도별 일사량 추이",
    color_discrete_map={
        "경북": "#d62728",  # 빨강
        "경남": "#1f77b4",  # 파랑
        "충남": "#2ca02c"   # 초록
    }
)

fig.update_traces(line=dict(width=3), marker=dict(size=9))

fig.update_layout(
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="일사량 (MJ/m²)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray", borderwidth=1)
)

fig.show()


In [135]:
pd.read_excel("dataset/2015-2024_충남,경북,경남_평균_일사량.xlsx")

,지역,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,충남,349,331,343,461,450,430,430,449,430,442
1,경북,441,432,459,483,467,457,447,478,470,463
2,경남,428,289,345,404,465,462,466,484,457,466


In [137]:
import pandas as pd
import plotly.express as px

# === 데이터 불러오기 ===
df = pd.read_excel("dataset/2015-2024_충남,경북,경남_평균_일사량.xlsx")

# === Wide → Long 변환 ===
df_long = df.melt(id_vars="지역", var_name="연도", value_name="일사량(MJ/m2)")
df_long["연도"] = pd.to_numeric(df_long["연도"], errors="coerce")

# === Plotly 선 그래프 ===
fig = px.line(
    df_long,
    x="연도",
    y="일사량(MJ/m2)",
    color="지역",
    markers=True,
    title="경북·경남·충남 2015~2024 평균 일사량 추이",
    category_orders={"지역": ["경북", "경남", "충남"]},  # 순서 강제
    color_discrete_map={
        "경남": "#1f77b4",  # 파랑
        "경북": "#d62728",  # 빨강
        "충남": "#2ca02c"   # 초록
    }
)

# 스타일 조정
fig.update_traces(line=dict(width=3), marker=dict(size=9))
fig.update_layout(
    template="plotly_white",
    xaxis=dict(dtick=1, title="연도"),
    yaxis=dict(title="일사량 (MJ/m²)"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray")
)

fig.show()


In [185]:
# 상단 import에 추가
import plotly.express as px

# -----------------------------------------------
# [NEW] 5) 데이터 모음 – 2024 버블 차트 생성 (사용자 로직 반영)
df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 버블 크기(설비용량 기반, 최소값 보정)
df_2024["설비용량"] = df_2024["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

color_map = {
    "서울": "#E74C3C","부산": "#3498DB","대구": "#2ECC71","인천": "#F39C12","광주": "#9B59B6",
    "대전": "#1ABC9C","울산": "#E67E22","경기": "#34495E","강원": "#F1C40F","경북": "#E91E63",
    "경남": "#00BCD4","전북": "#FF9800","전남": "#795548","충북": "#607D8B","충남": "#4CAF50","제주": "#673AB7"
}

fig = px.scatter(
    df_2024, x="일사량(MJ/m2)", y="설비이용률(%)",
    size="설비용량", color="지역", hover_name="지역", text="지역",
    size_max=60, title="2024년 지역별 일사량과 설비이용률 비교 분석",
    color_discrete_map=color_map
)
fig.update_traces(text=None)

THRESH = 500
for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        fig.add_annotation(x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
                            text=f"<b>{row['지역']}</b>", showarrow=False,
                            font=dict(color="white", size=11))
    else:
        fig.add_annotation(x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
                            text=row["지역"], showarrow=True, arrowhead=2,
                            ax=15, ay=-15, font=dict(color="black", size=11),
                            bgcolor="rgba(255,255,255,0.6)")

fig.update_layout(width=1300, height=800, template="plotly_white")
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady + 0.2], title="설비이용률 (%)")

# 👉 이 한 줄이 핵심: Plotly를 HTML 조각으로 추출
chart_html = fig.to_html(full_html=False, include_plotlyjs='cdn',
                            config={"displaylogo": False})


In [114]:
import plotly.express as px

# 2024년도 데이터만 선택
df_2024 = merged[merged["연도"] == 2024].copy()

# 막대그래프
fig = px.bar(
    df_2024,
    x="지역",
    y="일사량(MJ/m2)",
    text="일사량(MJ/m2)",
    color="일사량(MJ/m2)",   # 값에 따라 색상 구분
    color_continuous_scale="YlOrRd",  # 노랑~빨강 색상 스케일
    title="2024년 지역별 일사량 (MJ/m²)"
)

# 레이블, 서식 조정
fig.update_traces(texttemplate="%{text:.1f}", textposition="outside")
fig.update_layout(
    xaxis_title="지역",
    yaxis_title="일사량 (MJ/m²)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=80)
)
fig.update_xaxes(tickangle=-45)

fig.show()


In [108]:
import pandas as pd
import plotly.graph_objects as go

# 데이터 로드
df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 설비이용률 그래프
fig_cf = go.Figure()
fig_cf.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["설비이용률(%)"],
    text=[f"{v:.2f}%" for v in df_2024["설비이용률(%)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["설비이용률(%)"],
        colorscale="Oranges",
        showscale=True,
        colorbar=dict(title="설비이용률 (%)")
    ),
    hovertemplate="<b>%{x}</b><br>설비이용률: %{y:.2f}%<extra></extra>"
))

fig_cf.update_layout(
    title="2024년 지역별 설비이용률",
    xaxis_title="지역",
    yaxis_title="설비이용률 (%)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_cf.update_xaxes(tickangle=-45)
fig_cf.update_yaxes(tickformat=".2f")

fig_cf.show()


In [174]:
# 이용률 / 일사량 버블 차트 시각화 # 수정본

df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2024["설비용량"] = df_2024["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# color map 설정
color_map = {
    "서울": "#E74C3C",
    "부산": "#3498DB",
    "대구": "#2ECC71",
    "인천": "#F39C12",
    "광주": "#9B59B6",
    "대전": "#1ABC9C",
    "울산": "#E67E22",
    "경기": "#34495E",
    "강원": "#F1C40F",
    "경북": "#E91E63",
    "경남": "#00BCD4",
    "전북": "#FF9800",
    "전남": "#795548",
    "충북": "#607D8B",
    "충남": "#4CAF50",
    "제주": "#673AB7"
}

# 산점도
fig = px.scatter(
    df_2024,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="설비용량",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2024년 지역별 일사량과 설비이용률 비교 분석",
    color_discrete_map=color_map
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 크기 설정
fig.update_layout(width=1300,height=800)

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady + 0.2], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()

In [116]:
import pandas as pd
import plotly.express as px

df_2024 = merged[merged["연도"] == 2024].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2024["설비용량"] = df_2024["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# 산점도
fig = px.scatter(
    df_2024,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="설비용량",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2024년 지역별 일사량과 설비이용률 비교 분석"
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()


In [110]:
import pandas as pd
import plotly.express as px

df_2023 = merged[merged["연도"] == 2023].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2023["버블크기"] = df_2023["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2023["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2023["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# 산점도
fig = px.scatter(
    df_2023,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="버블크기",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2023년 지역별 일사량과 설비이용률 비교 분석"
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2023.iterrows():
    if row["버블크기"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()

In [111]:
import pandas as pd
import plotly.express as px

df_2022 = merged[merged["연도"] == 2022].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2022["버블크기"] = df_2022["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2022["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2022["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# 산점도
fig = px.scatter(
    df_2022,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="버블크기",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2022년 지역별 일사량과 설비이용률 비교 분석"
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2022.iterrows():
    if row["버블크기"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()

In [112]:
import pandas as pd
import plotly.express as px

df_2021 = merged[merged["연도"] == 2021].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2021["버블크기"] = df_2021["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2021["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2021["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# 산점도
fig = px.scatter(
    df_2021,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="버블크기",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2021년 지역별 일사량과 설비이용률 비교 분석"
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2021.iterrows():
    if row["버블크기"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()

In [84]:
import pandas as pd
import plotly.express as px

df_2020 = merged[merged["연도"] == 2020].copy()

# 버블 크기 (설비용량 기반, 최소값 보정)
df_2020["버블크기"] = df_2020["설비용량(MW)"].apply(lambda v: max(50, float(v)))

# 숫자형 변환
x = pd.to_numeric(df_2020["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2020["설비이용률(%)"], errors="coerce")

xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx = (xmax - xmin) * 0.05
pady = (ymax - ymin) * 0.05

mean_x = x.mean()
mean_y = y.mean()

# 산점도
fig = px.scatter(
    df_2020,
    x="일사량(MJ/m2)",
    y="설비이용률(%)",
    size="버블크기",
    color="지역",
    hover_name="지역",
    text="지역",
    size_max=60,
    title="2020년 지역별 일사량과 설비이용률 비교 분석"
)

# 기본 텍스트 제거 (조건부로 다시 추가)
fig.update_traces(text=None)

# 큰/작은 버블 라벨 처리
THRESH = 500  # 작은 버블 기준 (MW)

for _, row in df_2020.iterrows():
    if row["버블크기"] >= THRESH:
        # 큰 버블 → 중앙에 흰 글자(굵게 보이게 HTML 태그 사용)
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>",
            showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        # 작은 버블 → 바깥쪽 라벨 + 화살표
        fig.add_annotation(
            x=row["일사량(MJ/m2)"],
            y=row["설비이용률(%)"],
            text=row["지역"],
            showarrow=True, arrowhead=2,
            ax=15, ay=-15,
            font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax,
                line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y,
                line=dict(color="red", dash="dash", width=2))

# 사분면 라벨
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)",
                    showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.add_annotation(x=xmin, y=ymax, text="효율 우수 (자원↓, 효율↑)",
                    showarrow=False, font=dict(color="blue"), xanchor="left", yanchor="bottom")
fig.add_annotation(x=xmax, y=ymin, text="잠재력 有 (자원↑, 효율↓)",
                    showarrow=False, font=dict(color="orange"), xanchor="right", yanchor="top")
fig.add_annotation(x=xmin, y=ymin, text="저효율 (자원↓, 효율↓)",
                    showarrow=False, font=dict(color="grey"), xanchor="left", yanchor="top")

# 축 범위
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="합계 일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady], title="설비이용률 (%)")
fig.update_layout(template="plotly_white")

fig.show()